In [10]:
import pandas as pd
import numpy as np

# Load the two datasets into pandas dataframes

In [11]:
## Loading the dataset
data = np.loadtxt("data/HT_Sensor_dataset.dat", skiprows=1)

dataset = pd.DataFrame(
    {'id': data[:, 0],
     'time': data[:, 1],
     'r1': data[:, 2],
     'r2': data[:, 3],
     'r3': data[:, 4],
     'r4': data[:, 5],
     'r5': data[:, 6],
     'r6': data[:, 7],
     'r7': data[:, 8],
     'r8': data[:, 9],
     'temp': data[:, 10],
     'humidity': data[:, 11]
    })
dataset['id'] = dataset['id'].astype(int)

In [12]:
## Importing dataset
metadata = np.loadtxt("data/HT_Sensor_metadata.dat", skiprows=1, dtype=str)

metadataset = pd.DataFrame(
    {'id': metadata[:, 0],
     'date': metadata[:, 1],
     'class': metadata[:, 2],
     'start_time_hours': metadata[:, 3],
     'duration_hours': metadata[:, 4]
    })
metadataset['id'] = metadataset['id'].astype(int)

#Apparently no data
metadataset = metadataset[metadataset['id'] != 95]

## Inner Join the tables to get our main dataset

In [13]:
joined_df = pd.merge(dataset, metadataset, left_on='id', right_on='id', how='inner')
joined_df.info()

#Remove rows where time is less than zero
joined_df = joined_df[joined_df['time'] >= 0]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 928991 entries, 0 to 928990
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                928991 non-null  int64  
 1   time              928991 non-null  float64
 2   r1                928991 non-null  float64
 3   r2                928991 non-null  float64
 4   r3                928991 non-null  float64
 5   r4                928991 non-null  float64
 6   r5                928991 non-null  float64
 7   r6                928991 non-null  float64
 8   r7                928991 non-null  float64
 9   r8                928991 non-null  float64
 10  temp              928991 non-null  float64
 11  humidity          928991 non-null  float64
 12  date              928991 non-null  object 
 13  class             928991 non-null  object 
 14  start_time_hours  928991 non-null  object 
 15  duration_hours    928991 non-null  object 
dtypes: float64(11), int6

In [14]:
def change_class(row):
    if row['time'] < 0:
        return 'background'
    else:
        return row['class']

joined_df['class'] = joined_df.apply(change_class, axis=1)
joined_df.head()

,id,time,r1,r2,r3,r4,r5,r6,r7,r8,temp,humidity,date,class,start_time_hours,duration_hours
3429,0,0.000021,12.8102,10.3665,10.4529,11.6742,13.4941,13.2749,8.30531,9.04553,26.4234,59.4725,07-04-15,banana,13.49,1.64
3430,0,0.000309,12.8097,10.3655,10.4523,11.6734,13.4934,13.2740,8.30527,9.04545,26.4241,59.4745,07-04-15,banana,13.49,1.64
3431,0,0.000587,12.8088,10.3645,10.4516,11.6731,13.4930,13.2730,8.30523,9.04538,26.4246,59.4763,07-04-15,banana,13.49,1.64
3432,0,0.000865,12.8080,10.3638,10.4508,11.6727,13.4922,13.2719,8.30520,9.04516,26.4251,59.4779,07-04-15,banana,13.49,1.64
3433,0,0.001144,12.8078,10.3628,10.4503,11.6722,13.4914,13.2708,8.30517,9.04511,26.4256,59.4793,07-04-15,banana,13.49,1.64


In [15]:
X = []
y = []
## How many rows will be used for the prediction
snippet_size = 120
expected_rows_per_minute = 60
expected_snippet_time_gap = 1.05 * snippet_size / expected_rows_per_minute / 60
## How many rows will be skipped between each snippet
gap_size = 12
for start_index in range(0, len(joined_df) - snippet_size, gap_size):
    end_index = start_index + snippet_size
    ## Skip if the data spans across segments
    if joined_df.iloc[end_index,0] != joined_df.iloc[start_index,0]:
        continue
    ## If the time difference from beginning to end is larger than expected
    ## there must be some data missing or this chunk spans across tests
    if joined_df.iloc[end_index,1] - joined_df.iloc[start_index,1] > expected_snippet_time_gap:
        continue
    snippet = joined_df.iloc[start_index:end_index, 2:12].values
    X.append(snippet)
    y.append(joined_df.iloc[end_index,13])
    
X = np.array(X)
y = np.array(y)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train.shape)
print(y_train)

(37909, 120, 10)
['wine' 'wine' 'banana' ... 'background' 'background' 'banana']


In [17]:
from sklearn.metrics import classification_report

from sktime.classification.deep_learning.lstmfcn import LSTMFCNClassifier
import pickle

model = LSTMFCNClassifier(n_epochs=100, verbose=True)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
pred = model.predict(X_test)
print(classification_report(pred, y_test))
pickle.dump(model, open("trained_model", "wb"))

/Users/dapomeranz/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
2023-10-19 16:58:04.370633: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-10-19 16:58:04.370666: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-10-19 16:58:04.370675: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-10-19 16:58:04.370762: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-19 16:58:04.371105: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/tas

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 10, 120)]            0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 10, 128)              123008    ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 10, 128)              512       ['conv1d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 10, 128)              0         ['batch_normalization[0][0

2023-10-19 16:58:09.061003: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-19 16:58:09.750093: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-19 16:58:16.832107: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


297/297 [==============================] - 54s 126ms/step - loss: 0.6300 - accuracy: 0.7349
Epoch 2/100
297/297 [==============================] - 37s 125ms/step - loss: 0.4933 - accuracy: 0.7985
Epoch 3/100
297/297 [==============================] - 32s 109ms/step - loss: 0.4158 - accuracy: 0.8315
Epoch 4/100
297/297 [==============================] - 35s 117ms/step - loss: 0.3455 - accuracy: 0.8673
Epoch 5/100
297/297 [==============================] - 37s 125ms/step - loss: 0.2803 - accuracy: 0.8977
Epoch 6/100
297/297 [==============================] - 35s 117ms/step - loss: 0.2369 - accuracy: 0.9155
Epoch 7/100
297/297 [==============================] - 32s 106ms/step - loss: 0.2008 - accuracy: 0.9291
Epoch 8/100
297/297 [==============================] - 31s 104ms/step - loss: 0.1733 - accuracy: 0.9392
Epoch 9/100
297/297 [==============================] - 31s 104ms/step - loss: 0.1504 - accuracy: 0.9475
Epoch 10/100
297/297 [==============================] - 22s 75ms/step - loss

2023-10-19 18:01:38.186955: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-19 18:01:38.379861: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


75/75 [==============================] - 4s 19ms/step
0.9794260392487867
75/75 [==============================] - 1s 7ms/step
              precision    recall  f1-score   support

  background       1.00      0.97      0.98      2843
      banana       0.96      0.99      0.98      2981
        wine       0.98      0.98      0.98      3654

    accuracy                           0.98      9478
   macro avg       0.98      0.98      0.98      9478
weighted avg       0.98      0.98      0.98      9478

